In [ ]:
import json
import numpy as np
from matplotlib import pyplot as plt
from scipy.io import savemat

In [ ]:
pStr = lambda p : "{:04.0f}".format(p)

# File Paths (Parareal Solution)
PpDir = lambda k , pTot, p : f"nls-exp1/parareal-k-{k}P{pStr(pTot)}/Proc_{pStr(p)}"
ParSolFile = lambda k, pTot, p : f"{PpDir(k, pTot, p)}/yend.npy" 
ParTimingFile = lambda k, pTot, p : f"{PpDir(k, pTot, p)}/runtime.json" 

# File Paths (Serial RK - Fine)
ERKFinepDir = f"nls-exp1/serial-fineP{pStr(1)}/Proc_{pStr(0)}"
ERKFineSolFile = f"{ERKFinepDir}/yend.npy"
ERKFineTimingFile = f"{ERKFinepDir}/runtime.json"

# File Paths (Serial RK -- Reference)
ERKRefpDir = f"nls-exp1/serial-refP{pStr(1)}/Proc_{pStr(0)}"
ERKRefSolFile = f"{ERKRefpDir}/yend.npy"
ERKRefTimingFile = f"{ERKRefpDir}/runtime.json"

def extractTotalTime(timing_filepath):
    f = open(timing_filepath)
    data = json.load(f)
    f.close()
    return data["total"]

In [ ]:
# Experiment Parameters
ks = np.arange(0,6)
pTot = 1
pFin = pTot - 1

error_norm = lambda sol, ref : np.linalg.norm(sol - ref, np.inf) / np.linalg.norm(ref, np.inf)

In [ ]:
# Compute Errors 
y_ref = np.load(ERKRefSolFile)
y_fine = np.load(ERKFineSolFile)

errors_parareal = np.zeros(ks.size)
for i in range(0, ks.size):
    y_par = np.load(ParSolFile(ks[i], pTot, pFin))
    errors_parareal[i] = error_norm(y_par, y_ref)

error_fine = error_norm(y_fine, y_ref)

In [ ]:
# Compute Times
time_ref = extractTotalTime(ERKFineTimingFile)
time_fine = extractTotalTime(ERKFineTimingFile)

times_parareal = np.zeros(ks.size)
for i in range(0, ks.size):
    times_parareal[i] = extractTotalTime(ParTimingFile(ks[i], pTot, pFin))
    
speedup = time_fine / times_parareal;
savemat('nls-exp1-speedup', {"speedup" : speedup});

In [ ]:
plt.semilogy(ks, errors_parareal)
plt.semilogy(ks, error_fine * np.ones(ks.shape))
plt.show()

In [ ]:
plt.plot(ks, times_parareal)
plt.plot(ks, time_fine * np.ones(ks.shape))
plt.show()

In [ ]:
plt.plot(np.real(y_ref)) 
plt.show()